# Problem Statement and Description

In the Union Territory of New Delhi, there are a lot of places for the residents to go out and eat, and there are people who would like to take the opportunity of serving such people by opening his/her own restraunt. Food business always thrives on the quality of the food served, variety of the dishes and area in which it is operting. But, in large cities like New Delhi, it could be difficult to decide where should one open his new restraunt, after which he would have to decide which cuisines to offer and many more things. No worries, these problems could be easily solved by using the data, and applying certain tools on it.

So, the Final Problem is to-

1. Classify different areas in different Clusters
2. Decide where to open a restraunt

# Data Requirements

Since we have established a problem, we have to go ahead and decide what kinds of data would be needed. To solve the problem for deciding places where people usually go to eat, we have to determine that by clustering the neighborhoods into diiferent categories, and then determine which could be a more appropriate place to operate at. Then we would need the 

Important links:

1. https://simple.wikipedia.org/wiki/List_of_districts_in_Delhi
2. https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi

These are the sources used in identifying areas in Delhi for further segmentation, analysis and finding optimum place for a person to open a restaraunt.

Apart form this data, Foursquare API will be used to gather data regarding the places in each neighborhood, so that a more detailed and thorough analysis could be conducted by clustering different areas according to the types of places present in the locality and proper findings can be shown.

In [122]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

#### Load and explore the data

In [177]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup as bs
from sklearn.cluster import KMeans
import sys
import folium 

print('Libraries imported.')

Libraries imported.


In [2]:
url="https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi"
req=requests.get(url)

In [3]:
soup = bs(req.content,'lxml')

In [4]:
l={}
for item in soup.find_all('li')[:9]:
    l[item.text[2:]]=[]
keys=list(l.keys())

for item in soup.find_all('li')[12:30]:
    l[keys[0]].append(item.text)

for item in soup.find_all('li')[30:57]:
    l[keys[1]].append(item.text)

for item in soup.find_all('li')[57:71]:
    l[keys[2]].append(item.text)

for item in soup.find_all('li')[71:79]:
    l[keys[3]].append(item.text)

for item in soup.find_all('li')[79:89]:
    l[keys[4]].append(item.text)

for item in soup.find_all('li')[89:108]:
    l[keys[5]].append(item.text)

for item in soup.find_all('li')[108:157]:
    l[keys[6]].append(item.text)

for item in soup.find_all('li')[157:178]:
    l[keys[7]].append(item.text)

for item in soup.find_all('li')[178:198]:
    l[keys[8]].append(item.text)

In [6]:
res={'Districts':[],'Neighborhoods':[]}
for key in l.keys():
    dis=key
    for sub in l[key]:
        res['Districts'].append(dis)
        res['Neighborhoods'].append(sub)

In [8]:
pd.DataFrame(res)

,Districts,Neighborhoods
0,North West Delhi,Adarsh Nagar
1,North West Delhi,Ashok Vihar
2,North West Delhi,Azadpur
3,North West Delhi,Bawana
4,North West Delhi,Begum Pur
5,North West Delhi,Dhaka
6,North West Delhi,Jahangirpuri
7,North West Delhi,Karala
8,North West Delhi,Keshav Puram
9,North West Delhi,Kingsway Camp


Total 186 Neighborhoods are there in Delhi

In [30]:
res_add={'Latitude':[],'Longitude':[]}
tot=len(res['Districts'])
r=1
geolocator = Nominatim(user_agent="dl_explorer")
for dis,sub in zip(res["Districts"],res['Neighborhoods']):
    add=sub+" "+dis+" Delhi"
    lat=""
    max_at=0
    while lat=="":
        max_at+=1
        out=str(round(r/tot*100,2))+ " % "
        sys.stdout.write('\r'+out+sub)
        if max_at==10:
            break
        try: 
            location = geolocator.geocode(add)
            lat = location.latitude
            lng = location.longitude
            
        except: continue
    r+=1
    try:
        location = geolocator.geocode(sub+" Delhi")
        lat = location.latitude
        lng = location.longitude
    except:
        lat=0
        lng=0
    
    res_add['Latitude'].append(lat)
    res_add['Longitude'].append(lng)

100.0 % Vikaspuriargenuram

In [51]:
neighborhoods=pd.DataFrame({**res,**res_add})
neighborhoods.drop(neighborhoods[neighborhoods['Latitude']==0].index,axis=0,inplace=True)

neighborhoods

,Districts,Neighborhoods,Latitude,Longitude
0,North West Delhi,Adarsh Nagar,28.714401,77.167288
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Azadpur,28.707657,77.175547
3,North West Delhi,Bawana,28.799660,77.032885
5,North West Delhi,Dhaka,28.708698,77.205749
6,North West Delhi,Jahangirpuri,28.725972,77.162658
7,North West Delhi,Karala,28.735140,77.032511
8,North West Delhi,Keshav Puram,28.688926,77.161683
9,North West Delhi,Kingsway Camp,28.698778,77.204626
10,North West Delhi,Kohat Enclave,28.698041,77.140539


After removing areas for which geo data was unavailable, there are Total 172 Neighborhhods that will be used in further study

In [72]:
address = 'Delhi'

geolocator = Nominatim(user_agent="dl_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Delhi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New Delhi are 28.6517178, 77.2219388.


In [73]:
Delhi_Map=folium.Map(location=[latitude,longitude],zoom_start=10)
for sub,lat,lng in zip(neighborhoods['Neighborhoods'],neighborhoods['Latitude'],neighborhoods['Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=sub,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Delhi_Map)
Delhi_Map